In [1]:
from pathlib import Path
import sys, importlib, warnings
warnings.filterwarnings("ignore")

# repo root
cwd = Path().resolve()
root = None
for p in [cwd, *cwd.parents]:
    if (p/"src").is_dir():
        root = p; break
assert root
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

import pandas as pd
import numpy as np

import src.config, src.portfolio.optimizer, src.utils.plotting
importlib.reload(src.config); importlib.reload(src.portfolio.optimizer); importlib.reload(src.utils.plotting)

from src.config import Settings
from src.portfolio.optimizer import PortfolioOptimizer, PortfolioInputs
from src.utils.plotting import Plotter

cfg = Settings()
cfg

# Load processed features
features = pd.read_csv(cfg.data_processed_dir / "merged_features.csv", parse_dates=["Date"]).set_index("Date")

tickers = ["TSLA","BND","SPY"]
returns = features[[f"{t}_ret" for t in tickers]].dropna().copy()
returns.columns = tickers

# TSLA expected return from forecast (12m), fallback to historical mean if file missing
tsla_forecast_12m = (cfg.reports_figures_dir.parent / "interim" / "tsla_forecast_12m.csv")
tsla_forecast_12m = str(tsla_forecast_12m) if Path(tsla_forecast_12m).exists() else None

opt = PortfolioOptimizer(rf_rate=cfg.risk_free_rate)
exp_ann = opt.build_expected_returns(
    returns_df=returns.rename(columns={t: f"{t}_ret" for t in tickers}),
    use_tickers=tickers,
    tsla_forecast_csv=tsla_forecast_12m,
    tsla_mode="forecast_12m"
)

# Covariance from historical daily returns (annualized)
cov_ann = opt.build_covariance(
    returns_df=returns.rename(columns={t: f"{t}_ret" for t in tickers}),
    use_tickers=tickers
)

exp_ann, cov_ann.head()



(TSLA   -0.001124
 BND     0.019599
 SPY     0.144844
 dtype: float64,
           TSLA       BND       SPY
 TSLA  0.350331  0.001861  0.052950
 BND   0.001861  0.003016  0.001150
 SPY   0.052950  0.001150  0.033272)

In [2]:
inputs = PortfolioInputs(tickers=tickers, exp_returns_ann=exp_ann, cov_ann=cov_ann, rf_rate=cfg.risk_free_rate)

frontier = opt.efficient_frontier(inputs, n_points=60)
w_max, perf_max = opt.max_sharpe(inputs)       # (ret, vol, sharpe)
w_min, perf_min = opt.min_volatility(inputs)

print("Max Sharpe weights:", w_max)
print("Max Sharpe perf   (ret, vol, sharpe):", perf_max)
print("Min Vol   weights:", w_min)
print("Min Vol   perf    (ret, vol, sharpe):", perf_min)


Max Sharpe weights: OrderedDict({'TSLA': 0.0, 'BND': 0.0, 'SPY': 1.0})
Max Sharpe perf   (ret, vol, sharpe): (np.float64(0.14484446528686484), np.float64(0.18240716649610328), np.float64(0.6844274141473047))
Min Vol   weights: OrderedDict({'TSLA': 0.0, 'BND': 0.94511, 'SPY': 0.05489})
Min Vol   perf    (ret, vol, sharpe): (np.float64(0.026474116078194786), np.float64(0.05397824160633912), np.float64(0.11993936603956502))


In [3]:
pl = Plotter(cfg.reports_figures_dir)

frontier_fig = pl.efficient_frontier(
    frontier_df=frontier,
    maxpt=(perf_max[1], perf_max[0]),
    minvolpt=(perf_min[1], perf_min[0]),
    title="Efficient Frontier (TSLA-BND-SPY)",
    fname="efficient_frontier.png"
)

# Save weights & stats
out_dir = cfg.reports_figures_dir.parent / "interim"
out_dir.mkdir(parents=True, exist_ok=True)

pd.Series(w_max).to_csv(out_dir / "weights_max_sharpe.csv")
pd.Series(w_min).to_csv(out_dir / "weights_min_vol.csv")

stats_df = pd.DataFrame(
    {"portfolio":["max_sharpe","min_vol"],
     "exp_return":[perf_max[0], perf_min[0]],
     "volatility":[perf_max[1], perf_min[1]],
     "sharpe":[perf_max[2], perf_min[2]]}
)
stats_df.to_csv(out_dir / "portfolio_stats.csv", index=False)

frontier_fig, out_dir / "weights_max_sharpe.csv", out_dir / "weights_min_vol.csv", out_dir / "portfolio_stats.csv"


(WindowsPath('../reports/figures/efficient_frontier.png'),
 WindowsPath('../reports/interim/weights_max_sharpe.csv'),
 WindowsPath('../reports/interim/weights_min_vol.csv'),
 WindowsPath('../reports/interim/portfolio_stats.csv'))

In [4]:
# Example policy: recommend Max Sharpe unless volatility exceeds SPY's annualized vol by a threshold
spy_ann_vol = returns["SPY"].std() * np.sqrt(252)
choice = "max_sharpe"
if perf_max[1] > 1.5 * spy_ann_vol:
    choice = "min_vol"

print("Recommended:", choice)
print("Weights:\n", pd.read_csv(out_dir / f"weights_{choice}.csv", header=None, index_col=0).squeeze())
print("Stats:\n", stats_df.loc[stats_df["portfolio"] == choice])


Recommended: max_sharpe
Weights:
 0
NaN     0.0
TSLA    0.0
BND     0.0
SPY     1.0
Name: 1, dtype: float64
Stats:
     portfolio  exp_return  volatility    sharpe
0  max_sharpe    0.144844    0.182407  0.684427
